#Oil Extraction Production Forecasting
<br/>
<img src="https://www.nsenergybusiness.com/wp-content/uploads/sites/4/2022/07/refinery-ga56d4972f_640.jpg" />

In [0]:
#IMPORTANT! DO NOT CHANGE THESE TWO VALUES!!!!
catalog = "workshop"
db = "default"

#IMPORTANT! THIS NEEDS TO BE UNIQUE FOR EVERY PARTICIPANT!!!!
#IMPORTANT! THIS NEEDS TO BE THE NAME OF THE TABLE YOU CREATED FOR THIS LAB!!!!
src_table = "ademianczuk_oil_yield"

In [0]:
#Load the delta table into a PySpark dataframe
df = spark.table(f"{catalog}.{db}.{src_table}")

In [0]:
df.printSchema()
df.show(5, truncate=False)  # Display first 5 rows

In [0]:
display(df.describe())

In [0]:
# If any columns have missing values, we need to decide whether to fill, drop, or interpolate them. Sometimes empty or missing values may be valuable though.

from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

In [0]:
#Let's look for some seasonality based on the timeseries plot
import matplotlib.pyplot as plt
import pandas as pd

# Convert PySpark DataFrame to Pandas for plotting
df_pd = df.select("date", "yield_bbl").groupby("date").avg("yield_bbl").orderBy("date").toPandas()

# Plot time series
plt.figure(figsize=(12, 5))
plt.plot(df_pd["date"], df_pd["avg(yield_bbl)"], marker="o", linestyle="-")
plt.xlabel("Date")
plt.ylabel("Average Yield (BBL)")
plt.title("Oil Yield Trend Over Time")
plt.xticks(rotation=45)
plt.show()

In [0]:
import matplotlib.pyplot as plt
import pandas as pd

# Convert PySpark DataFrame to Pandas
df_pd = df.select("date", "temperature", "precipitation").toPandas()

# Convert date to datetime
df_pd["date"] = pd.to_datetime(df_pd["date"])

# Resample to weekly average to reduce data size
df_resampled = df_pd.set_index("date").resample("W").mean().reset_index()

# Create figure and axes
fig, ax1 = plt.subplots(figsize=(12, 5))

# Plot temperature on primary y-axis
ax1.plot(df_resampled["date"], df_resampled["temperature"], color="red", marker="o", linestyle="-", label="Temperature (°C)")
ax1.set_xlabel("Date")
ax1.set_ylabel("Temperature (°C)", color="red")
ax1.tick_params(axis="y", labelcolor="red")

# Create secondary y-axis for precipitation
ax2 = ax1.twinx()
ax2.bar(df_resampled["date"], df_resampled["precipitation"], color="blue", alpha=0.5, label="Precipitation (mm)")
ax2.set_ylabel("Precipitation (mm)", color="blue")
ax2.tick_params(axis="y", labelcolor="blue")

# Title and layout
plt.title("Temperature and Precipitation Over Time (Weekly Avg)")
fig.tight_layout()
plt.show()

In [0]:
# We need to check for abnormally high or low values in oil yield (barrels produced), well pressure and oil price.
# Convert to Pandas for visualization
df_outliers = df.select(["yield_bbl", "temperature", "well_pressure", "oil_price"]).toPandas()

# Plot boxplots
df_outliers.plot(kind="box", subplots=True, layout=(2, 2), figsize=(10, 8), sharex=False, sharey=False)
plt.suptitle("Box Plot of Key Features")
plt.show()

In [0]:
# Let's look for some field correlation
import seaborn as sns

# Convert PySpark DF to Pandas
df_corr = df.select(["yield_bbl", "temperature", "precipitation", "humidity", "wind_speed", "well_pressure", "sand_quality", "drilling_efficiency", "oil_price"]).toPandas()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_corr.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert PySpark DataFrame to Pandas
df_pd = df.select("yield_bbl", "temperature", "precipitation").toPandas()

# Create the KDE plot (bell curve)
plt.figure(figsize=(10, 6))

# Plot yield distribution
sns.kdeplot(df_pd["yield_bbl"], label="Yield (BBL)", color="red", linewidth=2)

# Labels and title
plt.xlabel("Value")
plt.ylabel("Density")
plt.title("Bell Curve of Yield")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert PySpark DataFrame to Pandas
df_pd = df.select("yield_bbl", "temperature", "precipitation").toPandas()

# Create the KDE plot (bell curve)
plt.figure(figsize=(10, 6))

# Plot yield distribution
sns.kdeplot(df_pd["temperature"], label="Temperature", color="green", linewidth=2)

# Labels and title
plt.xlabel("Value")
plt.ylabel("Density")
plt.title("Bell Curve of temperature")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert PySpark DataFrame to Pandas
df_pd = df.select("yield_bbl", "temperature", "precipitation").toPandas()

# Create the KDE plot (bell curve)
plt.figure(figsize=(10, 6))

# Plot yield distribution
sns.kdeplot(df_pd["precipitation"], label="Precipitation (mm)", color="blue", linewidth=2)

# Labels and title
plt.xlabel("Value")
plt.ylabel("Density")
plt.title("Bell Curve of precipitation")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

At this point we know that temperature and precipitation are our biggest contributing features to yield. We'll create a feature table with those values for forecasting.

Given that we want to boost our forecast with predicting temperature and precipitation, we'll likely also have to forecast those two features as well. The predictions of those fields will contribute to the prediction of yield. This means that we'll have to fix the distributions of yield and precipitation. The temperature looks okay since we know that seasonally there's a change between summer and winter with a degree of outliers.

## Evaluate Skew & Kurtosis

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Convert PySpark DataFrame to Pandas
df_pd = df.select("date","yield_bbl", "precipitation", "temperature").toPandas()

# Plot distributions
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df_pd["yield_bbl"], kde=True, bins=30, color="red")
plt.title("Yield Distribution")

plt.subplot(1, 3, 2)
sns.histplot(df_pd["precipitation"], kde=True, bins=30, color="blue")
plt.title("Precipitation Distribution")

plt.subplot(1, 3, 3)  
sns.histplot(df_pd["temperature"], kde=True, bins=30, color="green")
plt.title("Temperature Distribution")

plt.show()

In [0]:
from scipy.stats import skew, kurtosis

print(f"Yield Skewness: {skew(df_pd['yield_bbl'])}, Kurtosis: {kurtosis(df_pd['yield_bbl'])}")
print(f"Precipitation Skewness: {skew(df_pd['precipitation'])}, Kurtosis: {kurtosis(df_pd['precipitation'])}")
print(f"Temperature Skewness: {skew(df_pd['temperature'])}, Kurtosis: {kurtosis(df_pd['temperature'])}")

All three will require a boxcox transformation & normalization on using a standard scale

## When to apply transformations
Before storing features or after?

We're going to do our initial investigation to see the effect of a box-cox transformation, however we'll be capturing our features and creating our feature tables pre-transformation. We want to encapsulate this kind of transformation in the model itself so we don't have to tightly couple the transformation with the feature engineering and ingestion process.

We will do a sample transformation below to see the effect of the box-cox transformation and its effect on our data set.

In [0]:
from scipy.stats import boxcox, yeojohnson

df_pd["yield_bbl"], lambda_yield = boxcox(df_pd["yield_bbl"] + 1)  # Shift to avoid zero
df_pd["precipitation"], lambda_precip = boxcox(df_pd["precipitation"] + 1)
df_pd["temperature"], lambda_temp = yeojohnson(df_pd["temperature"] + 1)

print(f"Box-Cox Lambda for Yield: {lambda_yield}")
print(f"Box-Cox Lambda for Precipitation: {lambda_precip}")
print(f"Yeo-Johnson Lambda for Temperature: {lambda_temp}")

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()  # Use MinMaxScaler() if you prefer [0,1] range

df_pd[["yield_bbl", "precipitation", "temperature"]] = scaler.fit_transform(df_pd[["yield_bbl", "precipitation", "temperature"]])

In [0]:
df_pd

In [0]:
# Plot distributions
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df_pd["yield_bbl"], kde=True, bins=30, color="red")
plt.title("Yield Distribution")

plt.subplot(1, 3, 2)
sns.histplot(df_pd["precipitation"], kde=True, bins=30, color="blue")
plt.title("Precipitation Distribution")

plt.subplot(1, 3, 3)  
sns.histplot(df_pd["temperature"], kde=True, bins=30, color="green")
plt.title("Temperature Distribution")

plt.show()

In [0]:
# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_pd.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

In [0]:
# Plot boxplots
df_pd.plot(kind="box", subplots=True, layout=(2, 2), figsize=(10, 8), sharex=False, sharey=False)
plt.suptitle("Box Plot of Key Features")
plt.show()

In [0]:
# Convert date to datetime
df_pd["date"] = pd.to_datetime(df_pd["date"])

# Resample to weekly average to reduce data size
df_resampled = df_pd.set_index("date").resample("W").mean().reset_index()

# Create figure and axes
fig, ax1 = plt.subplots(figsize=(12, 5))

# Plot temperature on primary y-axis
ax1.plot(df_resampled["date"], df_resampled["temperature"], color="red", marker="o", linestyle="-", label="Temperature (°C)")
ax1.set_xlabel("Date")
ax1.set_ylabel("Temperature (°C)", color="red")
ax1.tick_params(axis="y", labelcolor="red")

# Create secondary y-axis for precipitation
ax2 = ax1.twinx()
ax2.bar(df_resampled["date"], df_resampled["precipitation"], color="blue", alpha=0.5, label="Precipitation (mm)")
ax2.set_ylabel("Precipitation (mm)", color="blue")
ax2.tick_params(axis="y", labelcolor="blue")

# Title and layout
plt.title("Temperature and Precipitation Over Time (Weekly Avg)")
fig.tight_layout()
plt.show()

So based on this, I'm pretty satisfied with temperature and precipitation as potentially relevant features to predict yield. We'll go back to our 'raw' state and create a feature table based on those two numeric fields along our date.

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

#Load our desired features, along with a monotonically increasing id. Feature tables require a unique identifier.
df_features = df.select("date", "temperature", "precipitation", "yield_bbl").withColumn("id", monotonically_increasing_id())

Now let's create our feature table. We'll be using this feature table for our experiment and training our model. Creating a feature table ensures discoverability and consistency when using this data for modelling and training.

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient

fe = FeatureEngineeringClient()

# Create feature table with `id` as the primary key.
customer_feature_table = fe.create_table(
  name=f'{catalog}.{db}.{src_table}_features',
  primary_keys=['id', 'date'],
  schema=df_features.schema,
  description='oil yield features',
  df = df_features,
  timeseries_columns='date'
)